# RAG: Поиск в интернете (Web Search)

В этом ноутбуке мы создадим агента, который использует **инструмент Web Search** для поиска актуальной информации в интернете.

## Задача

Создать сводку новостей об агентах искусственного интеллекта в 2025 году.

## Архитектура

```
┌─────────────────┐      ┌─────────────────┐      ┌─────────────────┐
│  Jupyter Client │──────│   Yandex Cloud  │──────│    Yandex       │
│  (этот ноутбук) │      │   Responses API │      │    Search       │
└─────────────────┘      └─────────────────┘      └─────────────────┘
```

Responses API автоматически вызывает Yandex Search для получения актуальной информации.

Для начала, установим необходимые библиотеки:

In [ ]:
%pip install --quiet openai python-dotenv

> **ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

## Авторизация

Для работы с Yandex Cloud нам понадобится `folder_id` (идентификатор каталога) и `api_key` (API-ключ сервисного аккаунта). Мы предполагаем, что эти значения хранятся в файле `.env` в текущей директории, который можно скачать, выполнив ячейку ниже (убедитесь предварительно, что в ней указан правильный адрес для скачивания):

In [ ]:
!curl -o .env {{url_of_dotenv_file}}

In [4]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]
model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

def printx(string):
    display(Markdown(string))

print(f"✅ Авторизация настроена (folder_id: {folder_id[:8]}...)")

✅ Авторизация настроена (folder_id: b1gbicod...)


---

## Часть 1: Импорт класса Agent

В этом примере мы будем использовать класс `Agent`, который мы рассмотрели в предыдущем разделе применительно к вызову инструментов. Класс Agent позволяет указать список инструментов, которые сможет использовать агент, в виде Pydantic-классов (для локальных инструментов), или же в виде JSON-описания, которое используется для задания внутренних инструментов из Responses API.

Импортируем этот класс из файла [Agent.py](Agent.py):

In [2]:
from Agent import Agent, create_client

client = create_client()
print("✅ Клиент создан")

✅ Клиент создан


---

## Часть 2: Настройка инструмента Web Search

Инструмент `web_search` позволяет модели искать актуальную информацию в интернете. Для использования инструмента достаточно задать его в виде сдоварного описания такого вида: 


In [3]:
# Инструмент для поиска в интернете
web_search_tool = {
    "type": "web_search",
    "search_context_size": "high",  # Больше контекста для детального обзора
}

print("✅ Инструмент Web Search настроен")

✅ Инструмент Web Search настроен


### Доступные параметры:
- `type`: "web_search"
- `search_context_size`: объём контекста — "low", "medium", "high"
- `filters.allowed_domains`: список доменов (до 5 шт.)
- `filters.user_location.region`: код региона для локализации

Попробуем использовать инструмент поиска с Responses API:

In [21]:
res = client.responses.create(
    model = model,
    instructions = "Ты - умный помощник. Используй поиск в интернет в случае необходимости",
    tools = [ web_search_tool ],
    input = "В каких публикациях 2026 года упоминается про использование агентов для медицинской диагностики?"
)

printx(res.output_text)

На основе представленных материалов, в публикациях 2026 года упоминается использование агентов для медицинской диагностики и в более широких аспектах здравоохранения. Вот ключевые источники:

1. **Дзен-издание SIM lab.** — в статье ["2026: здравоохранение восемь прорывных технологических трендов"](https://dzen.ru/a/aUkWr_SpOUYBqgEG) от 22 декабря 2025 года:
   - Раздел **"2. AI-агенты в здравоохранении"** посвящён именно этому вопросу.
   - Упоминается, что в 2026 году медицинские организации начнут активно использовать **AI-агентов** для сопровождения пациента на всём пути — от первичного опроса симптомов до анализа результатов, выявления отклонений и контроля лечения.
   - Агенты смогут выполнять сложные многоэтапные процессы, взаимодействовать с внешними системами и помогать в диагностике, анализе данных и профилактике заболеваний.

2. **Журнал Nature (по версии Science.Mail.ru)** — в обзоре ["Самые ожидаемые научные события 2026 года"](https://science.mail.ru/news/41655-samye-ozhidaemye-nauchnye-sobytiya-2026-goda-nature/) от 20 декабря 2025 года:
   - В разделе **"ИИ как соавтор открытий"** указано, что в 2026 году ожидается широкое использование **ИИ-агентов**, объединяющих несколько языковых моделей для выполнения сложных процессов.
   - Прогнозируется, что такие агенты могут привести к **первым значимым научным прорывам в медицине**, включая диагностику.

3. **Forbes** — в статье ["ИИ-агенты в медицине: новый этап цифровой трансформации системы здравоохранения"](https://blogs.forbes.ru/2025/11/20/ii-agenty-v-medicine-novyj-jetap-cifrovoj-transformacii-sistemy-zdravoohranenija/) от 20 ноября 2025 года:
   - Хотя публикация вышла в 2025 году, она прогнозирует развитие событий **до 2030 года**, включая 2026 год.
   - Отмечается, что ИИ-агенты уже способны с высокой точностью анализировать медицинские изображения (например, легочные узелки и рак молочной железы).
   - Ожидается, что к 2030 году более 60% рутинных задач, включая **анализ инструментальных исследований**, будут делегированы ИИ.

Таким образом, хотя прямых публикаций **строго за 2026 год** в представленных результатах нет, **несколько авторитетных источников, публиковавшихся в конце 2025 года, прямо указывают на 2026 год как на ключевой период для внедрения ИИ-агентов в медицинскую диагностику и здравоохранение в целом.**

Посмотрим внимательнее на структуру ответа:

In [19]:
print(f"Длина ответа: {len(res.output)}")
for a in res.output:
    print(f" - {a.type}")

Длина ответа: 2
 - web_search_call
 - message


В ответе присутствует отдельная запись для вызова инструмента поиска. Из неё, в частности, мы можем извлечь использованный поисковый запрос:

In [20]:
res.output[0].action.query

'{"query": "публикации 2026 года использование агентов для медицинской диагностики", "lang": "ru"}'

Можем ограничить диапазон поиска только доменом arxiv.org:

In [26]:
web_search_arxiv = {
    "type": "web_search",
    "search_context_size": "high",
    "filters" : {
        "allowed_domains": [ "arxiv.org" ]
    }
}

res = client.responses.create(
    model = model,
    instructions = """Ты - умный помощник. Используй инструмент поиска в 
    интернет в случае необходимости.""",
    tools = [ web_search_arxiv ],
    input = "В каких публикациях 2026 года упоминается про использование агентов для медицинской диагностики?"
)
print(f"Поисковый запрос: {res.output[0].action.query}")
printx(res.output_text)

Поисковый запрос: {"query": "публикации 2026 года использование агентов для медицинской диагностики", "lang": "ru"}


На основе представленных публикаций 2026 года, использование агентов для медицинской диагностики рассматривается в следующих работах:

---

### 1. **[2602.06980] "Potential Role of Agentic Artificial Intelligence in Toxicologic Pathology"**
- **Авторы**: Nasir Rajpoot, Richard Haworth, Xavier Palazzi и др.
- **Ссылка**: [https://arxiv.org/abs/2602.06980](https://arxiv.org/abs/2602.06980)
- **Краткое содержание**:
  - Исследуется применение **агентного ИИ (agentic AI)** в токсикологической патологии — важной области медицинской диагностики.
  - Предлагается система из нескольких специализированных агентов, которые совместно:
    - Собирают данные (гистопатология, клинические анализы, литература),
    - Интегрируют информацию,
    - Генерируют черновик отчёта с цитированием источников,
    - Обеспечивают прозрачность и следуемость выводов.
  - Подчёркивается, что **врач остаётся в центре процесса** — агенты лишь помогают, но не заменяют эксперта.
  - Обсуждаются барьеры внедрения: доверие, валидация, безопасность данных, соответствие нормативным требованиям (например, ALCOA++ и GLP).

---

### 2. **[2602.08013] "Small Agent Group is the Future of Digital Health"**
- **Авторы**: Yuqiao Meng, Luoxi Tang, Dazheng Zhang и др.
- **Ссылка**: [https://arxiv.org/pdf/2602.08013](https://arxiv.org/pdf/2602.08013)
- **Краткое содержание**:
  - Предлагается концепция **Small Agent Group (SAG)** — группы небольших ИИ-агентов, имитирующих команду врачей.
  - В отличие от одного большого языкового модели (LLM), SAG:
    - Распределяет задачи между агентами: **рассуждение, поиск знаний, проверка безопасности, синтез решения**.
    - Использует многоагентные дебаты для критики и уточнения выводов, что снижает риск галлюцинаций.
  - Показано, что SAG превосходит одиночные LLM по:
    - **Точности диагностики**,
    - **Надёжности и последовательности выводов**,
    - **Снижению демографической предвзятости**.
  - Подчёркивается важность **этических и безопасных решений** в клинической практике.

---

### 3. **[2602.08254] "SynthAgent: A Multi-Agent LLM Framework for Realistic Patient Simulation"**
- **Авторы**: Arman Aghaee, Sepehr Asgarian, Jouhyun Jeon
- **Ссылка**: [https://arxiv.org/abs/2602.08254](https://arxiv.org/abs/2602.08254)
- **Краткое содержание**:
  - Представлена система **SynthAgent** для моделирования виртуальных пациентов с ожирением и сопутствующими психическими расстройствами.
  - Используются **многоагентные ИИ**, имитирующие поведение пациентов, их реакцию на лечение и прогрессирование болезни.
  - Применяется для:
    - Исследования поведенческих паттернов,
    - Оценки приверженности лечению,
    - Прогнозирования эффективности терапии.
  - Позволяет проводить исследования без риска для пациентов и нарушения конфиденциальности.

---

### Общие тенденции:
1. **Многоагентные системы** заменяют или дополняют одиночные LLM.
2. Агенты специализируются на отдельных задачах: диагностика, безопасность, интерпретация данных.
3. Акцент на **прозрачность, проверяемость и этику** — особенно важно для медицины.
4. **Человек-в-петле (human-in-the-loop)** — врач сохраняет контроль над финальным решением.
5. Системы тестируются на **реальных клинических сценариях** и показывают превосходство над традиционными подходами.

---

### Вывод:
В 2026 году агентные ИИ активно исследуются как инструменты поддержки медицинской диагностики, особенно в сложных, междисциплинарных случаях. Основной фокус — на **безопасность, надёжность и интерпретируемость**, а не просто на автоматизацию.

Здесь мы применяли поиск для одного конкретного запроса, но далее для удобства будем использовать поиск как один из инструментов класса `Agent`.

---

## Часть 3: Создание агента-исследователя

Создадим агента, который будет искать и анализировать новости об AI-агентах:

In [ ]:
researcher_instruction = """
Ты — аналитик-исследователь в области искусственного интеллекта.

Твоя задача — искать в интернете актуальную информацию и составлять 
структурированные аналитические обзоры.

При составлении обзора:
1. Используй только факты из найденных источников
2. Указывай ключевые события и тренды
3. Приводи названия компаний и продуктов
4. Выделяй важные даты
5. Делай выводы на основе данных

Формат ответа:
- Заголовок
- Основные события и тренды
- Ключевые игроки
- Выводы

Пиши на русском языке.
"""

news_agent = Agent(
    client=client,
    instruction=researcher_instruction,
    tools=[web_search_tool],
    tool_choice="auto"
)

print("🔍 Агент-исследователь готов!")

🔍 Агент-исследователь готов!


Обратите внимание, что мы написали подробный промпт для агента, указывая ему примерное содержание и состав ответа, который мы ожидаем получить. Это направляет агента на поиск в более правильном направлении.

---

## Часть 4: Исследование новостей об AI-агентах в 2025 году

In [27]:
# Запрос на обзор новостей
result = news_agent(
    "Сделай обзор главных новостей об AI-агентах (AI Agents) в 2025 году. "
    "Какие компании запустили новые продукты? Какие тренды появились? "
    "Что изменилось в области автономных AI-систем?"
)

printx(result.output_text)

## Обзор главных новостей об AI-агентах в 2025 году

### Основные события и тренды:
- **Рост популярности автономных ИИ-агентов:** на смену простым чат-ботам и ассистентам приходят системы, способные самостоятельно планировать и выполнять сложные задачи. Согласно Artificial Intelligence Index Report 2025, 78% компаний уже используют ИИ-решения.
- **Развитие мультиагентных систем:** несколько ИИ-агентов взаимодействуют между собой. Например, на автоматизированных складах беспилотные погрузчики координируют маршруты.
- **Внедрение ИИ-агентов в различные отрасли:**
  - в здравоохранении платформа Agentforce for Health от Salesforce автоматизирует административные функции;
  - в логистике решения Oracle анализируют цепочки поставок и оптимизируют логистику;
  - в финансовом секторе система Holly AI от Trade Ideas анализирует фондовый рынок;
  - в маркетинге и аналитике платформа Dynamic Yield (принадлежит Mastercard) адаптирует контент веб-сайтов под каждого пользователя.
- **Появление новых стандартов и протоколов:** например, Model Context Protocol (MCP) стал стандартом интеграции ИИ-агентов с внешними системами. Его поддержали крупные компании, включая OpenAI, Google DeepMind, Microsoft и AWS.
- **Увеличение внимания к вопросам безопасности:** OWASP выделяет несколько ключевых угроз, связанных с использованием ИИ-агентов, включая отравление памяти, неправомерное использование инструментов и каскадные галлюцинации.
- **Экологические аспекты использования ИИ:** исследование Университета Калифорнии показало, что тренировка модели уровня GPT-5 требует огромного количества воды для охлаждения. В ЕС уже рассматривают возможность введения «водного налога» для дата-центров.
- **Развитие инструментов для создания ИИ-агентов:** появились визуальные платформы автоматизации (например, n8n) и фреймворки для разработки мультиагентных систем (например, CrewAI, LangGraph, AutoGen от Microsoft).

### Ключевые игроки:
- **Anthropic:** представила Claude 3.5 Sonnet, который научился взаимодействовать с компьютерными интерфейсами как человек (например, нажимать кнопки, вводить текст, управлять курсором).
- **OpenAI:** представила агента Operator для автоматизации повседневных задач в интернете.
- **Яндекс:** встроил в свой браузер первого в России бесплатного ИИ-агента на базе Алисы.
- **Microsoft:** работает над интеграцией агентских технологий в продукты семейства Copilot.
- **Oracle, Salesforce:** внедряют ИИ-агентов в свои решения для бизнеса.
- **Trade Ideas:** использует систему Holly AI в финансовом секторе.
- **Стартапы и проекты:** Deep Research (агент-исследователь), Manus (мультиагентная система), Genspark (агент-секретарь), AutoGPT (фреймворк для разработчиков), Browser Operator (агент для работы в браузере), Project Astra от Google (универсальный помощник).

### Выводы:
- ИИ-агенты переходят от экспериментальных проектов к массовому внедрению в бизнес-процессы и повседневную жизнь.
- Наблюдается тренд на интеграцию ИИ-агентов в различные отрасли — от здравоохранения до финансов и логистики.
- Растёт внимание к вопросам безопасности, этическим аспектам и экологическим последствиям использования ИИ-агентов.
- Развитие мультиагентных систем и стандартизация интеграции (MCP) открывают новые возможности для расширения применения ИИ-агентов.
- Вместе с возможностями ИИ-агенты приносят и новые риски, требующие разработки строгих нормативных стандартов и защитных механизмов.
- В ближайшие годы ИИ-агенты продолжат проникать в корпоративные и пользовательские сервисы, изменяя подходы к автоматизации процессов и управлению данными.
- Несмотря на значительные успехи, существуют ограничения и проблемы, связанные с пониманием ИИ-агентами причинно-следственных связей и «внутренней моделью мира». Это приводит к случаям «провалов» и неэффективной работы в сложных или нестандартных ситуациях.

---

## Часть 5: Дополнительные примеры

In [28]:
# Поиск с ограничением по доменам
tech_news_tool = {
    "type": "web_search",
    "search_context_size": "medium",
    "filters": {
        "allowed_domains": ["habr.com", "vc.ru", "tadviser.ru"]
    }
}

habr_agent = Agent(
    client=client,
    instruction="Ты собираешь новости с русскоязычных IT-ресурсов. Пиши кратко.",
    tools=[tech_news_tool]
)

result = habr_agent("Какие новости про LLM были на Хабре в последнее время?")
printx(result.output_text)

На Хабре недавно были опубликованы следующие статьи, связанные с LLM:

1. **«Итоги LLM в 2025 году: прогресс, проблемы и прогнозы»**. В статье представлен обзор ключевых достижений в области больших языковых моделей (LLM) за 2025 год. Среди основных тем:
   - выпуск модели DeepSeek R1, которая продемонстрировала возможности развития способности к рассуждениям с помощью обучения с подкреплением;
   - обсуждение методов RLVR (Reinforcement Learning with Verifiable Rewards) и GRPO, которые позволили улучшить LLM на этапе постобучения;
   - анализ стоимости обучения моделей и выводы о том, что оно может быть значительно дешевле, чем предполагалось ранее;
   - перспективы развития LLM, включая расширение применения RLVR в различных областях и продолжение работы над масштабированием инференса.

2. **«Шесть сдвигов в LLM за 2025 год — итоги от Андрея Карпати»**. Андрей Карпати, один из основателей OpenAI, выделил несколько ключевых изменений в области LLM в 2025 году:
   - главным техническим событием года он называет RLVR — обучение с подкреплением на задачах с автоматически проверяемыми ответами (математика и код);
   - концептуальный сдвиг: LLM рассматриваются не как «растущие организмы», а как «призраки», которых мы вызываем, с «рваным» интеллектом;
   - появление нового класса продуктов, работающих поверх LLM, которые занимаются контекст-инжинирингом и оркестрируют множество вызовов модели;
   - новая парадигма взаимодействия с ИИ: вместо облачных контейнеров — локальные приложения;
   - термин «vibe coding» — программирование через человеческий язык без погружения в код;
   - перспективы GUI для LLM: будущее за генерацией изображений, инфографики, анимаций и интерактивных приложений.

3. В статье о новостях инфобеза также упоминаются некоторые аспекты, связанные с LLM:
   - в январе 2026 года были обнаружены расширения для Chrome, которые крадут чаты с LLM;
   - нарастает усталость пользователей от повсеместной интеграции LLM, появляются скрипты для удаления ИИ-фич из Windows 11;
   - есть слухи, что Microsoft станет менее агрессивно внедрять LLM в свои продукты.

In [29]:
# Продолжение диалога (контекст сохраняется)
result = news_agent("Расскажи подробнее про один из трендов, который ты упомянул")
printx(result.output_text)

### Развитие мультиагентных систем

**Описание тренда:**
Мультиагентные системы — это комплекс, включающий несколько ИИ-агентов, которые могут взаимодействовать между собой для решения сложных задач. Каждый агент в системе может выполнять определённые функции и при необходимости координировать свои действия с другими агентами. Это позволяет решать задачи, которые сложно выполнить одному агенту, и повышает общую эффективность системы.

**Примеры применения:**
- **Автоматизированные склады:** беспилотные погрузчики, управляемые ИИ-агентами, координируют свои маршруты и действия для оптимизации процессов складирования и отгрузки товаров. Агенты обмениваются данными о местоположении товаров, состоянии складских зон и необходимых операциях, что позволяет сократить время обработки заказов и минимизировать ошибки.
- **Логистика и транспорт:** мультиагентные системы могут оптимизировать маршруты доставки, учитывая различные факторы, такие как трафик, погодные условия, состояние дорог и загруженность транспортных узлов. Агенты, отвечающие за разные аспекты логистики, взаимодействуют между собой и с внешними системами (например, с системами отслеживания грузов), чтобы обеспечить максимально эффективное выполнение задач.
- **Финансовые рынки:** в трейдинге и управлении активами мультиагентные системы могут анализировать рынок с разных точек зрения, учитывать различные факторы (новости, экономические показатели, исторические данные) и принимать более обоснованные решения. Агенты могут специализироваться на определённых типах активов или стратегиях, взаимодействуя для достижения общей цели — максимизации прибыли и минимизации рисков.
- **Кибербезопасность:** агенты могут отслеживать различные аспекты сетевой безопасности, обнаруживать угрозы и координировать свои действия для их нейтрализации. Например, один агент может отвечать за мониторинг сетевого трафика, другой — за анализ уязвимостей в программном обеспечении, а третий — за реагирование на инциденты.

**Технологии и инструменты:**
Для разработки и внедрения мультиагентных систем используются различные технологии и инструменты:
- **Фреймворки для разработки мультиагентных систем:** например, CrewAI, LangGraph, AutoGen от Microsoft. Эти инструменты предоставляют разработчикам необходимые средства для создания, тестирования и развёртывания мультиагентных систем.
- **Протоколы и стандарты интеграции:** например, Model Context Protocol (MCP) позволяет интегрировать ИИ-агентов с внешними системами и обеспечивать их взаимодействие. Поддержка крупных компаний (OpenAI, Google DeepMind, Microsoft, AWS) способствует распространению и стандартизации таких протоколов.
- **Платформы для автоматизации и интеграции:** визуальные платформы, такие как n8n, позволяют разработчикам быстро создавать и настраивать рабочие процессы, включая взаимодействие между различными ИИ-агентами.

**Преимущества мультиагентных систем:**
- **Повышенная эффективность:** распределение задач между несколькими агентами позволяет решать сложные проблемы быстрее и эффективнее.
- **Гибкость и масштабируемость:** системы легко адаптируются к изменяющимся условиям и могут масштабироваться в зависимости от потребностей бизнеса.
- **Улучшение качества решений:** взаимодействие между агентами позволяет учитывать больше факторов и принимать более обоснованные и точные решения.

**Вызовы и проблемы:**
- **Сложность разработки и настройки:** создание и настройка мультиагентных систем требуют значительных усилий и знаний. Необходимо учитывать множество факторов, включая совместимость агентов, их взаимодействие и влияние на общую производительность системы.
- **Вопросы безопасности и надёжности:** взаимодействие между агентами и внешними системами увеличивает риски, связанные с безопасностью данных и надёжностью работы системы. Необходимо разрабатывать строгие механизмы защиты и контроля.
- **Этические и правовые аспекты:** использование мультиагентных систем в различных областях поднимает вопросы ответственности, прозрачности и соответствия законодательным требованиям.

**Перспективы развития:**
Мультиагентные системы имеют большой потенциал для применения в различных отраслях. Их дальнейшее развитие будет способствовать созданию более умных и эффективных систем, способных решать сложные задачи в реальном времени и адаптироваться к изменяющимся условиям. Стандартизация и развитие инструментов для разработки таких систем будут играть ключевую роль в их распространении и внедрении в бизнес-процессы.

---

## Часть 6: Агент для мониторинга конкурентов

In [31]:
competitor_instruction = """
Ты — аналитик конкурентной разведки.

Задача: собирать информацию о продуктах и активности компаний.

В ответе указывай:
- Название компании
- Последние новости и анонсы
- Новые продукты или функции
- Оценку активности (высокая/средняя/низкая)
"""

competitor_agent = Agent(
    client=client,
    instruction=competitor_instruction,
    tools=[web_search_tool]
)

result = competitor_agent("Что нового у OpenAI и Anthropic в области AI-агентов?")
printx(result.output_text)

**OpenAI**

**Последние новости и анонсы:**
* OpenAI запустила ИИ-агента Operator, который способен взаимодействовать с веб-страницами, выполняя задачи без использования API сайтов.
* Представлена корпоративная платформа Frontier, которая позволяет «нанять ИИ-коллег» для выполнения различных задач.
* Выпущена модель GPT-5.3-Codex, ориентированная на «агентный» стиль программирования, способная выполнять задачи целиком: от понимания проблемы до готового результата.
* OpenAI представила десктопное приложение Codex для macOS.
* Запущен инструмент Prism для написания и проверки научных текстов.
* Внедрена система прогнозирования возраста пользователей ChatGPT для ограничения доступа к чувствительному контенту.
* Добавлена функция «Ваш год с ChatGPT» — интерактивный годовой обзор использования чат-бота.
* Выпущена новая версия ChatGPT Images — инструмента для генерации и редактирования изображений.
* Запущены групповые чаты в ChatGPT.
* Представлена модель GPT-5.1 с двумя режимами работы: Instant и Thinking.
* Добавлена функция «Обновить» в ChatGPT, позволяющая вмешиваться в процесс генерации ответа.

**Новые продукты и функции:**
* Operator — ИИ-агент для взаимодействия с веб-страницами.
* Responses API — новый API для создания AI-агентов, заменяющий API Assistants.
* Codex для macOS — десктопное приложение.
* Prism — инструмент для написания и проверки научных текстов.
* Функция «Ваш год с ChatGPT».
* ChatGPT Images — инструмент для генерации и редактирования изображений.
* Групповые чаты в ChatGPT.
* GPT-5.1 — новая версия модели с двумя режимами работы.
* Функция «Обновить» в ChatGPT.

**Оценка активности:** высокая.

**Anthropic**

**Последние новости и анонсы:**
* Anthropic выпустила обновлённую флагманскую модель Claude Opus 4.6 с функцией «команды агентов» в среде для разработчиков Claude Code.
* Компания заявила, что Claude останется без рекламы, объяснив несовместимость рекламных стимулов с действительно полезным ИИ-ассистентом.
* Anthropic объявила о партнёрстве с NASA — ИИ помог роботу Perseverance проехать 400 метров на Марсе.
* Представлен Claude Haiku 4.5 с возможностями кодирования и высокой скоростью работы.
* Anthropic сотрудничает с Allen Institute и Howard Hughes Medical Institute для ускорения научных открытий.
* ServiceNow выбрала Claude для создания клиентских приложений и повышения внутренней производительности.
* Anthropic и правительство Великобритании запустили проект по внедрению ИИ-помощи на сайте GOV.UK.
* Компания привлекла 30 млрд долларов в рамках очередного раунда финансирования, рыночная стоимость Anthropic выросла до 380 млрд долларов.
* Реклама Anthropic во время Super Bowl способствовала росту посещаемости сайта и количества активных пользователей Claude.

**Новые продукты и функции:**
* Claude Opus 4.6 — обновлённая модель с функцией «команды агентов».
* Claude Code — среда для разработчиков с возможностью разбиения крупных задач на части и запуска нескольких ИИ-субагентов.
* Плагины Claude Cowork для помощи бизнесу с аналитикой финансовых данных, юридическими инструментами, продажами и другими функциями.
* Юридический инструмент для автоматизации проверки контрактов и юридических консультаций.

**Оценка активности:** высокая.

---

## Заключение

Мы научились вооружать наших агентов инструментом веб-поиска `web_search`, чтобы они могли исследовать интернет. Это может быть полезно во многих случаях для снижения галлюцинаций, получения актуальной информации и подведение итогов по поисковым выдачам.